### Import libraries and data

In [ ]:
import pandas as pd
import csv
import re
import numpy as np

In [ ]:
vols = pd.read_csv("volumes.csv")
vols = vols.drop("Unnamed: 0", axis=1)

In [ ]:
temps = pd.read_csv("temperatures.csv")
temps = temps.drop("Unnamed: 0", axis=1)

In [ ]:
unitformula = pd.read_csv("scaledformulae.csv")
unitformula = unitformula.drop("Unnamed: 0", axis=1)

### Setting models

In [ ]:
from sklearn import linear_model, svm, kernel_ridge
regVols = linear_model.LinearRegression()
regAlpha = linear_model.LinearRegression()
#regVols = linear_model.ElasticNet(positive = True)

### Percentage function

In [ ]:
def percCalc (row):
    predictedvolume = row["sum"]
    vols = row["Volume"]
    percentage = predictedvolume / vols
    if (percentage >=0.95) and (percentage <=1.05):
        return 1
    else:
        return 0

The function removes any predicted volumes above or below 5% of the actual volume, before the forthcoming iteration

### Loop algorithm

In [ ]:
alpha = 0.00
elements = unitformula.columns.values
unitformula["Keep"] = 1
weights = []
alphas =[]
for chunk in zip(unitformula.groupby(np.arange(len(unitformula)) // 10000),vols.groupby(np.arange(len(vols)) // 10000),temps.groupby(np.arange(len(temps)) // 10000)):
    print "Chunk: ", chunk[0][0]
    prevalpha = 1
    chunkformula = chunk[0][1]
    chunkvol = chunk[1][1]
    chunktemp = chunk[2][1]
    keepidx = chunkformula.index.tolist()
    i = 0
    while abs(prevalpha-alpha) > 0.00001 :
        print "Iteration:", i
        regVols.fit(chunkformula[chunkformula.Keep == 1], chunkvol.loc[keepidx].div(chunktemp.loc[keepidx].Temperature*alpha+1, axis="index"))
        w = regVols.coef_
        x2=pd.DataFrame()
        for line in zip(elements, w[0]):
            x2[line[0]] = chunk[0][1][line[0]].apply(lambda x: x*line[1])
        x2["sum"] = x2.sum(axis=1)
        vol2 = chunkvol.loc[keepidx].div(x2["sum"].loc[keepidx], axis="index") - 1
        regAlpha.fit(chunktemp.loc[keepidx], vol2)
        if i > 0:
            prevalpha = alpha
            #print "Previous Alpha: ", prevalpha
        alpha = regAlpha.coef_[0][0]
        #print "Alpha: ", alpha
        chunkformula["Keep"] = pd.concat([x2,vols], axis=1).apply(percCalc, axis=1)
        keepidx = chunkformula.Keep[chunkformula.Keep == 1].index.tolist()
        #print "Element average volume (cubic angstroms)", zip(elements,w[0])
        i +=1
        if (len(keepidx) <= 1) or (i > 200):
            print "Failure"
            break
    weights.append(w[0])
    alphas.append(alpha)

Before the loop begins the data is seperated into chunks (size of a 10000), and empty arrays are declared to store required data.

The loop is similar to the previous ones, by successive iteration the average element volumes and alpha values are solved - the stopping criteria for the loop currently is where the difference in the absolute alpha values (from one iteration to the next) is below 0.00001.

For each iteration the percentage function above is applied and those that abide by the %5 rule are kept for the next iteration (below the entries which are used are those that have "1" in the "keep" column).

As a failsafe if the loop doesn't stop after 200 iterations or if 0 entries are kept for the next iteration, the loop for the current chunk stops, this is noted by the output "Failure".

### Misc. - Printing items to check.

In [ ]:
print alphas

In [ ]:
print chunkformula.index.tolist()

In [ ]:
elements = unitformula.columns.values
print chunkformula[elements][chunkformula.Keep == 1]